In [2]:
import numpy as np
from sklearn.ensemble import IsolationForest
from tensorflow import keras

In [3]:
class CorrectionLearner:
    def __init__(self):
        self.memory = []
        self.model = self._init_model()
        
    def _init_model(self):
        """Neural network to learn from manual overrides"""
        model = keras.Sequential([
            keras.layers.Dense(16, activation='relu', input_shape=(6,)),
            keras.layers.Dense(8, activation='relu'),
            keras.layers.Dense(3, activation='softmax')  # left/right/stop
        ])
        model.compile(optimizer='adam', loss='categorical_crossentropy')
        return model
    
    def record_correction(self, sensors, user_action):
        """
        sensors: Dictionary of sensor readings when correction occurred
        user_action: 'left', 'right', or 'stop'
        """
        self.memory.append({
            'tof': sensors['tof'],
            'ultrasonic': sensors['ultrasonic'],
            'speed': sensors['speed'],
            'action': user_action
        })
        
        # Retrain periodically
        if len(self.memory) % 50 == 0:
            self._retrain_model()
    
    def _retrain_model(self):
        """Update model with new correction data"""
        X = []
        y = []
        
        for record in self.memory:
            X.append([
                record['tof']/8191.0,
                record['ultrasonic']/400.0,
                record['speed']/2.0,
                record['tof_prev']/8191.0 if 'tof_prev' in record else 0,
                record['ultrasonic_prev']/400.0 if 'ultrasonic_prev' in record else 0,
                record['speed_prev']/2.0 if 'speed_prev' in record else 0
            ])
            
            action = record['action']
            y.append([1,0,0] if action == 'left' else
                    [0,1,0] if action == 'right' else
                    [0,0,1])
        
        self.model.fit(np.array(X), np.array(y), epochs=10, verbose=0)